In [1]:
!pip install --upgrade dagshub mlflow scikit-learn scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 58.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 95.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 82.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: 

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [3]:
import dagshub
dagshub.init(repo_owner='nmach22', repo_name='Fraud-Detection-ml', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=4a14124b-66f1-4c12-9186-d2b7d42fdcc7&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=bd4b0e7fe48f98ba47a111cbaad00c516b5a0a8e597f676ec445620a1e72c044




Output()

Accessing as nmach22

Initialized MLflow to track repo "nmach22/Fraud-Detection-ml"

Repository nmach22/Fraud-Detection-ml initialized!

In [4]:
test_transaction_df = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')

In [5]:
test_identity_df = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')

In [6]:
df = pd.merge(test_transaction_df, test_identity_df, on='TransactionID', how='left')

In [15]:
df.columns = df.columns.str.replace('^id-', 'id_', regex=True)

In [16]:
df.columns

Index(['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD',
       'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=433)

In [17]:
import mlflow
logged_model = 'runs:/0dc9f8feae2943e59332e32a47dafd60/pipeline'

# Load model as a SKlearn.
loaded_model = mlflow.sklearn.load_model(logged_model)

# Predict on a Pandas DataFrame.
preds_test_subm = loaded_model.predict_proba(df)

In [18]:
submission = pd.DataFrame({
    'TransactionID': test_transaction_df['TransactionID'],
    'isFraud': preds_test_subm[:, 1]
})

# Save to CSV
submission.to_csv('Submission_Logistic_regression_imblearner_random_undersampler_with probabilities.csv', index=False)

In [19]:
submission

,TransactionID,isFraud
0,3663549,0.023615
1,3663550,0.062311
2,3663551,0.190723
3,3663552,0.040715
4,3663553,0.035730
...,...,...
506686,4170235,0.149217
506687,4170236,0.273913
506688,4170237,0.066651
506689,4170238,0.095329


## Submission Fraud/Not Fraud distribution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_class_distribution(y, title="Class Distribution"):
    counts = y.value_counts()
    plt.figure(figsize=(6,4))
    sns.barplot(x=counts.index, y=counts.values, palette="viridis")
    plt.title(title)
    plt.xticks([0, 1], ['Not Fraud (0)', 'Fraud (1)'])
    plt.ylabel("Number of Samples")
    plt.xlabel("Class")
    for i, count in enumerate(counts.values):
        plt.text(i, count + 100, f'{count:,}', ha='center')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_class_distribution(submission['isFraud'])

In [ ]:
print(submission['isFraud'].sum() / len(submission))